In [6]:
import numpy as np
import GPy
from scripts.process import ContactPointTracker

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initializing Sensor Model

In [7]:
# Two ways to initialize a sensor model

# one is to pass the calibration file to the ContactPointTracker as follows
var = 1.0
R = 100

kern = GPy.kern.ThinPlate(3, variance=var, R=R)
tracker = ContactPointTracker(fm_alpha=1.04, fm_stride=1, sigma_process_init=5, mu0=np.array([-10,-20,0]))
tracker.init_calibration_model(calibration_fn='../data/all_datasets/calibration/tip_calib_May_04.csv',
                            input_ref=np.array([[64.30451325,98.0687005,16.1023935]]),
                            kernel=kern)

# Tracking on Reference Shapes

In [21]:

import plotly.graph_objects as go

# Initialize sensor model
var = 1.0
R = 100
kern = GPy.kern.ThinPlate(3, variance=var, R=R)
tracker = ContactPointTracker(fm_alpha=1.04, fm_stride=1, sigma_process_init=5, mu0=np.array([-10,-20,0]))
tracker.init_calibration_model(calibration_fn='../data/all_datasets/calibration/tip_calib_May_04.csv',
                            input_ref=np.array([[64.30451325,98.0687005,16.1023935]]),
                            kernel=kern)

tracking_data = {}
fig = go.Figure()

for i in range(0,1):
    tracker.initialize(track_data_fn=f'../data/all_datasets/test/tip_test_05_25_23/reference_cylindertip_test_scan_object_May_25_trial{i}.csv',
                                offset_mode='from_data')

    x_ukf, sig_ukf, additional_data = tracker.run_tracker(track_mode='static')

    # （3, n) to (n, 3), the estimated contact position at each timestep
    x_est = x_ukf.T
    x_est[:,2] = -x_est[:,2]
    # x_est is the estimated contact position in sensor frame
    x_err = additional_data['X_t']-x_est

    # filter the data that is making contact
    selection_arr_np = np.where(np.array(additional_data['selection_arr']))[0]
    x_err_sel = x_err[selection_arr_np,:]

    sig_ukf_sel = sig_ukf[:,:,selection_arr_np]

    # calculate the eiganvalues of the covariance matrix at each timestep
    ev_array = np.zeros(sig_ukf_sel.shape[2])
    for j in range(sig_ukf_sel.shape[2]):
        # obtain the eiganvalues of position x to itself
        ev_array[j] = np.linalg.eig(sig_ukf_sel[:,:,j])[0][1]
    # filter the data with eiganvalues that is less than 1, means converging
    x_err_filt = x_err_sel[np.where(ev_array < 1)[0],:]

    tracking_data[f'traj_{i}'] = x_err_filt
    fig.add_trace(go.Scatter3d(x=x_err_filt[15:-1,0], y=x_err_filt[15:-1,1],z=x_err_filt[15:-1,2],
                        name='lines', marker=dict(size=3)))

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.5, y=-1.5, z=0.1)
)
fig.update_layout(title=f'3D Localization Thin Plate Model var={var} R={R}', 
                scene = dict(
                    xaxis = dict(nticks=4, range=[-30,70],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
                    yaxis = dict(nticks=4, range=[-40,80],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
                    zaxis = dict(nticks=4, range=[10,-90],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
                    aspectratio=dict(x=1, y=1, z=1)),
                xaxis_title='x axis (mm)', yaxis_title='y axis (mm)',
                autosize=False,
                width=500, height=500,
                scene_camera=camera,
                margin=dict(l=10, r=10, b=10, t=50))

fig.show()
fig.write_image("exp_sandbox/traces.svg")

[5.00000000e+00 1.35657041e-01 6.96367988e-03 2.55986346e-03
 1.62203574e-03 8.98778964e-04 7.35604168e-04 6.30369122e-04
 5.53184533e-04 4.96768886e-04 4.53114356e-04 4.19016353e-04
 3.91739930e-04 3.69926012e-04 3.52317610e-04 3.37806820e-04
 3.25805396e-04 3.15981722e-04 3.07676334e-04 3.00633807e-04
 2.94201065e-04 2.88482789e-04 2.83433598e-04 2.78988195e-04
 2.75000995e-04 2.71519104e-04 2.68443496e-04 2.65764038e-04
 2.63455162e-04 2.61321007e-04 2.59341619e-04 2.57573186e-04
 2.56036537e-04 2.54661598e-04 2.53419465e-04 2.52242805e-04
 2.51179095e-04 2.50220585e-04 2.49345324e-04 2.48612218e-04
 2.47999512e-04 2.47445741e-04 2.46967995e-04 2.46487732e-04
 2.45986488e-04 2.45443132e-04 2.44889553e-04 2.44333270e-04
 2.43793791e-04 2.43280475e-04 2.42781413e-04 2.42332531e-04
 2.41904480e-04 2.41525981e-04 2.41208074e-04 2.40914626e-04
 2.40591239e-04 2.40269920e-04 2.39971119e-04 2.39688500e-04
 2.39427497e-04 2.39205246e-04 2.39035584e-04 2.38915530e-04
 2.38858196e-04 2.388193